In [1240]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "compar.": "comparative",
    "compar": "comparative",
    "impér.": "imperative",
    "impér": "imperative",
    "diminutif": "diminutive"
}

In [1131]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [1241]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [1442]:
REST = {
    170: "gʲλbg̤ʹαb (giubgeab) « bavardage, jacassement » ; kʲλtə (ciota) « chope » ; kʲλkə (ciuca) « lequel des deux » ; dʲλks (diucs)"# dans des jurons : hɑnəm ɩnʹ dʲλks (th’ anam ó’n diucs) « le diable t’emporte » ; fʲλlər (fiolar) « aigle » ; flʲλχ (fliuch) « humide » ; gʲλks (giucs) « un mot, une syllabe » ; gʲλlə (giolla) « groom, garçon de ferme » ; glʲλgɩrʹɩ (gliogaire) « vantard, fumiste » ; pʲλk (piuc) « une petite quantité, rien du tout » ; pʲλbər (piobar) « poivre » ; ʃλk et ʃʲλk (siuc) « gel » ; ʃλpə (siopa) « boutique », sə tʲλpə (insan tsiopa) « dans la boutique » ; sb̬ʹrʹλk (spriuc) « pierre levée »",
}

ɩnʲλməd ou ən ᴜməd (an iomad) « une grande quantité » ; ᴜmədu꞉ⁱlʹ (iomadamhail) « abondant » ; ɩnʲ λmərkə ou ən ᴜmərkə (an iomarcadh) « trop » ; go hᴜmərkəχ (go hiomarcach) « de grand cœur » ; λnəs ou ᴜnəs (ionas) « façon d’être » ; ɩ nʲλnəs (i n‑ionas) « de façon… », pris absolument : « de façon égale » ; ᴜməl (imeall) « bordure » ; ə nᴜməl ou ɩ nʲλməl (i n‑imeall) « à côté de »



bʹrʹeh ou bʹrʹᴇh (breith) « se saisir de » ; vʹeh ou vʹᴇh (bheith) « être » ; tʹeh ou tʹᴇh (te) « chaud » ;  bʹehirʹ (beithir) « bête sauvage » ; bʹrʹehəv (breith­eamh) « juge » ; klʹehɩrʹɩ fʹirʹ (cleithire fir) « homme grand et gros »


§ 213. 
tnɑ̃꞉hɩmʹ (tnáthaim) « je fatigue » ; tlɑ꞉h (tláth) « aimable, courtois »

§ 215. Groupes de deux consonnes : constitués

pʹrʹαb (preab) « sursaut » ; srɑ꞉ⁱdʹ (sráid) « rue » ; flahʹ (flaith) « prince » ; ɩ mʹlʹiꞏənə (i mbliadhna) « cette année » ; ǥɑ꞉ χlᴜꞏɛʃ (dhá chluais) « deux oreilles » ; də hlɑ̃꞉ⁱnʹtʹɩ (do shláinte) « ta santé » ; knɑ̃꞉v (cnámh) « os » ; gno꞉ (gnó) « affaire » ; mnɑ̃꞉ (mná), gén. et plur. de bʹαn (bean) « femme » ; ʃnʹi꞉ᵊv (sníomh) « filer » : do ǥno꞉ (do ghnó) « ton affaire » ; ɑnəhnɑ̃꞉v (anashnámh) « nage impec­cable ».

smʹɩgʹi꞉nʹ (smigín) « menton ».

stɑdɩmʹ (stadaim) « j’arrête ».

§ 216. Groupes de trois consonnes : Sont toujours consti­tués par sifflante + occlusive sourde ou assourdie + liquide.

sb̬ʹrʹλk (spriuc) « but, point de repère » ; stʹrʹi꞉ᵊpəχ ou ʃtʹrʹi꞉ᵊpəχ (stríopach) « putain » ; sg̬raʃtʹɩ (scraiste) « vaurien » ; sb̬lᴀᴜŋk (splannc) « éclair ».

§ 217. Les groupes implosifs n’ont jamais plus de deux consonnes. Ils peuvent être consti­tués par :

kɔrp (corp) « corps » ; kʹαrt (ceart) « droit » ; kʹᴇlʹtʹ (ceilt) « cacher » (pour la pronon­ciation de tʹ dans ce cas, cf. § 30) ; kʹαrk (cearc) « poule » ; kʹirʹkʹ (circ), dat. du même mot ; kalʹkʹ (cailc) « craie » ; kʲɑ꞉rd (ceárd) « métier » ; sg̬ɑrf (scarf) « écharpe ».





In [1443]:
# DATA[section] = []
section = 170
if not section in DATA:
    DATA[section] = []
extract(REST[section], ";", 1)

 dʲλks (diucs)
4


In [1444]:
def split_trans(pos):
    if " ou " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" ou ")
    elif " et " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" et ")

# split_trans(-1)
for s in range(len(DATA[section])):
    if "transcription" in DATA[section][s]:
        split_trans(s)

In [1447]:
import json

if str(section) in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [1446]:
# DATA[section] = DATA[section][0:5]
# del(DATA[section][-1])
# DATA[section][-1]["from"] = "prás"
# DATA[section][13]["in_phrase"] = "ceirt ar gach aon taobh"
# DATA[section][0]["contrast"] = "ᴜmədu꞉ⁱlʹ"
# DATA[section][0]["contrast_id"] = "167_2"
# DATA[section][2]["contrast"] = "go hᴜmərkəχ"
# DATA[section][2]["contrast_id"] = "167_4"
# DATA[section][-2]["specific_to"] = "Dunquin"
DATA[section].append(
    {
      "section": 170,
      "id": "170_4",
      "transcription": "dʲλks",
      "irish": "diucs",
      "french_note": "dans des jurons",
      "in_phrase": "th’ anam ó’n diucs",
      "see_id": "170_5"
    })